IMPORTS

In [1]:
pip install streamlit plotly pandas

Note: you may need to restart the kernel to use updated packages.


Create the Dashboard.py

In [2]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime
import pickle
import json

# Loading the prediction 
def load_model():
    with open("models/best_model.pkl", 'rb') as f:
        model = pickle.load(f)
    with open("models/model_metadata.json", 'r') as f:
        metadata = json.load(f)
    return model, metadata

# predict function
def predict(features, model, metadata):
    expected_features = [f for f in metadata['features'] if f not in ['Date', 'next_day_change']]
    input_df = pd.DataFrame([features])[expected_features]
    pred = model.predict(input_df)[0]
    return {
        'predicted_change': float(pred),
        'direction': 'UP' if pred > 0 else 'DOWN',
        'confidence': min(99, abs(pred)*100)
    }

# Dashboard UI
def main():
    st.set_page_config(page_title="Stock Predictor", layout="wide")
    
    # Load the model 
    model, metadata = load_model()
    
    st.title("📊 Stock Movement Predictor")
    st.subheader("Real-time market predictions using ML")
    
    # Sidebar for inputs
    with st.sidebar:
        st.header("Input Features")
        features = {
            'High': st.number_input("High Price", value=255.50),
            'Low': st.number_input("Low Price", value=248.20),
            'Open': st.number_input("Open Price", value=252.30),
            'Close': st.number_input("Close Price", value=250.00),
            'Volume': st.number_input("Volume", value=5000000),
            'daily_return': st.number_input("Daily Return", value=0.015),
            'avg_sentiment': st.slider("Avg Sentiment", -1.0, 1.0, 0.15),
            'tweet_count': st.number_input("Tweet Count", value=85),
            '5_day_ma': st.number_input("5-Day MA", value=250.50),
            '10_day_volatility': st.number_input("10-Day Volatility", value=0.02),
            'sentiment_lag1': st.number_input("Sentiment Lag (1)", value=0.12),
            'sentiment_ma3': st.number_input("Sentiment MA (3)", value=0.14)
        }
    
    # Main content area
    col1, col2 = st.columns(2)
    
    with col1:
        if st.button("Predict Movement", type="primary"):
            with st.spinner("Analyzing market trends..."):
                try:
                    result = predict(features, model, metadata)
                    
                    st.metric("Predicted Change", 
                            f"{result['predicted_change']:.2f}%",
                            delta=f"{result['direction']} ({result['confidence']:.1f}% confidence)")
                    
                    # Visualize prediction
                    fig = px.bar(x=["Prediction"], 
                                y=[abs(result['predicted_change'])], 
                                color=[result['direction']],
                                color_discrete_map={'UP':'green','DOWN':'red'},
                                labels={'y':'Magnitude (%)', 'x':''},
                                title="Movement Prediction")
                    st.plotly_chart(fig, use_container_width=True)
                    
                except Exception as e:
                    st.error(f"Prediction failed: {str(e)}")
    
    with col2:
        st.subheader("Feature Importance")
        # (Add your feature importance visualization if available)
        st.info("Feature importance visualization would go here")
        
        st.subheader("Recent Predictions")
        # (Add prediction history if persisting to database)
        st.info("Prediction history would appear here")

if __name__ == "__main__":
    main()

2025-04-20 19:12:44.271 
  command:

    streamlit run c:\Users\DELL\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
